In [1]:
from bs4 import BeautifulSoup
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import base64
import imagehash
import mysql.connector
import os
import re

In [2]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_anarchia"


# == Files config ==
MAIN_FOLDER = r"D:\dnmarchives\dnmarchives\anarchia"

In [3]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

In [4]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_anarchia ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  price VARCHAR(50),"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  has_image VARCHAR(25) NOT NULL DEFAULT 'has_image',"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [5]:
def update_image(product_id, value):
    query = "UPDATE products_tmp_anarchia SET has_image = %s WHERE id = %s"
    cursor.execute(query, (value, product_id))
    conn.commit()

In [6]:
def get_content(content):
    clear_content = ""

    if content is not None:
        raw_c = content.text

        clear_content = re.sub(r'[\t\r\n]', ' ', raw_c)
        clear_content = re.sub(' +',' ', clear_content).lstrip()
        
    return clear_content

In [7]:
p_img_hashes = {}
Image.MAX_IMAGE_PIXELS = None

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\index.php\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                content = page_content.find("table", attrs={"class": "itemInfo"})

                if content is not None:
                    desc_content = content.find("div", attrs={"class": "well"})

                    if desc_content is not None:
                        description = get_content(desc_content)

                    ## GET TITLE
                    title = ""
                    title_content = content.find("span", attrs={"class": "btn"})

                    if title_content is not None:
                        title = get_content(title_content)

                    ## GET SELLER
                    seller = ""
                    seller_content = content.find("tbody")

                    if seller_content is not None:
                        seller_content = seller_content.findAll("td")[0]
                        
                        if seller_content is not None:
                            seller_content = seller_content.find("a")
                            
                            if seller_content is not None:
                                seller = get_content(seller_content)

                    if description != "" or title != "":
#                         # Insert product into database
                        query = "INSERT INTO products_tmp_anarchia (name, description, seller_name, market_name, has_image, illegal) VALUES (%s, %s, %s, %s, %s, %s)"
                        values = (title, description, seller, "Anarchia", "no_image", "t")

                        cursor.execute(query, values)
                        conn.commit()

                        product_id = cursor.lastrowid


                        ## GET IMAGE
                        raw_data = page_content.find("div", attrs={"id": "images"})

                        if raw_data is not None:
                            raw_data = raw_data.find("a")
                            
                            if raw_data is not None:
                                img_path = raw_data["href"]
                                name = img_path.replace("http://tkmlejimlt72jnkm.onion/index.php/", "")

                                if name is not "":
                                    url_image = Path(f"{folder_image}\index.php\{name}")
                                    
                                    print(f"{product_id} - {url_image}")

#                                     # copy images to new directory, renaming with product id
#                                     if url_image.exists():
#                                         print(f"Image {product_id} exists... creating")
#                                         dst_dir = Path(f"D:\images\_anarchia\{product_id}.jpeg")

#                                         with open(url_image) as img_file:
#                                             data = img_file.read()
#                                             imgc_data = "" + data.replace("data:image/jpeg;base64,", "")
#                                             img_data = base64.b64decode(imgc_data)

#                                             with open(dst_dir, 'wb') as f:
#                                                 f.write(img_data)
#                                                 f.close
#                                             img_file.close
# #                                             copyfile(url_image, dst_dir)
# #                                             update_image(product_id, "has_image")
#                                     else:
#                                         print(f"Image {product_id} does not exists")
#                                 else:
#                                     print(f"Image {product_id} has no name")

1 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\d66d33d8f1d780c7
2 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\08fef66c0fd4a34f
3 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\0f6bc8c24d19114b
4 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\025466df87195981
5 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\906641954a3584eb
6 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\bd88e3a678861c94
7 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\527b486fe5f46093
8 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\c323efc75801e4f8
9 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\bb2f31da64d8596b
10 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\40cc3af77f0b4586
11 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\9a086f40ff5e41ac
12 - D:\dnmarchives\dnmarchives\anarchia\2015-05-07\index.php\image\e68c1a

100 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\fc87b536e3cc837a
101 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\2074f5cf7697d69d
102 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\a5a6b0abfdf045f7
103 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\1fe3baef58e78d33
104 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\a407a91325f69c36
105 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\07c8c82d9fac2a17
106 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\41ddfbca06a06da3
107 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\b30be94c4fc8f2d6
108 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\9373361f83521a7a
109 - D:\dnmarchives\dnmarchives\anarchia\2015-05-10\index.php\image\d5d5329d2f47d7f0
110 - D:\dnmarchives\dnmarchives\anarchia\2015-05-15\index.php\image\d66d33d8f1d780c7
111 - D:\dnmarchives\dnmarchives\anarchia\2015-05-15\i

201 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\222e7db832d08d6a
202 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\38b7c84864152a33
203 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\cd28b499217f2f6f
204 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\68397391a885d80f
205 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\a2acc76965b12da4
206 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\5af6eaca7e09c3ab
207 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\844e570e0a4e37c9
208 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\226f08183bef1ae5
209 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\cd9a666407e40cde
210 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\d69e997048f20e5e
211 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\index.php\image\79f209baa6c485c0
212 - D:\dnmarchives\dnmarchives\anarchia\2015-05-16\i

300 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\0091777e8c226e9b
301 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\4cf68a4dac7f3e9e
302 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\344119b60a5fca9e
303 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\30ea936cc104ccee
304 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\7ea84213e139a980
305 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\7555e1d78c63ff87
306 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\b7753c9eaf4410fa
307 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\9fc45aec289d8c17
308 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\9405b7cb2b80c582
309 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\43e18237cff37091
310 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\index.php\image\e48cde5fce46be3f
311 - D:\dnmarchives\dnmarchives\anarchia\2015-05-20\i

398 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\7ea84213e139a980
399 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\7555e1d78c63ff87
400 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\b7753c9eaf4410fa
401 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\9fc45aec289d8c17
402 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\9405b7cb2b80c582
403 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\43e18237cff37091
404 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\e48cde5fce46be3f
405 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\99c187853f515a2d
406 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\65b9e8d303c463a5
407 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\a8e7766398852627
408 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\index.php\image\caed29c200dc046d
409 - D:\dnmarchives\dnmarchives\anarchia\2015-05-24\i

...

1463 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\14474024b719c150
1464 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\344119b60a5fca9e
1465 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\249c543434f73128
1466 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\30ea936cc104ccee
1467 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\7ea84213e139a980
1468 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\50ed5d5ddf6b823a
1469 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\c6b601d57661222c
1470 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\b7753c9eaf4410fa
1471 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\6a413e6c2717e994
1472 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\68fe4aca534f6250
1473 - D:\dnmarchives\dnmarchives\anarchia\2015-06-22\index.php\image\9405b7cb2b80c582
1474 - D:\dnmarchives\dnmarchives\anarchia\

1561 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\4735e21ff524a30d
1562 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\6f9d655b0d639db3
1563 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\94803aacd54daa9a
1564 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\204f7f56e56a7463
1565 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\48ae125da45b16ea
1566 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\decc0700901c37bc
1567 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\37ba29175b636beb
1568 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\1c8399c4199678a9
1569 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\cd9a666407e40cde
1570 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\49f1acbb9d89ffe3
1571 - D:\dnmarchives\dnmarchives\anarchia\2015-06-28\index.php\image\d69e997048f20e5e
1572 - D:\dnmarchives\dnmarchives\anarchia\

1658 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\9a086f40ff5e41ac
1659 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\33d55e2c1e5dab06
1660 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\7f062953990e84d8
1661 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\459f403a2ad25748
1662 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\1caa685f312ac7f9
1663 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\0a05dd601f5c6fff
1664 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\036f9282e0d7157f
1665 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\06984a82d263136f
1666 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\25f90a6526caa2c4
1667 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\222e7db832d08d6a
1668 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\3f9e608e7fa1c5ae
1669 - D:\dnmarchives\dnmarchives\anarchia\

1754 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\9278ac8cfe72d2ae
1755 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\0c2b348936e365e2
1756 - D:\dnmarchives\dnmarchives\anarchia\2015-07-03\index.php\image\d5d5329d2f47d7f0
1757 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\d66d33d8f1d780c7
1758 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\08fef66c0fd4a34f
1759 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\5b27c36498030f1d
1760 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\cc29168d33813922
1761 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\3f6538d881632296
1762 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\7f7ecb06a6bd412e
1763 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\1f837dbc222bf2f9
1764 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\517029573b32a84f
1765 - D:\dnmarchives\dnmarchives\anarchia\

1855 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\5eed312721210718
1856 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\0ff35d2e142072a1
1857 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\6c13ec0ff5eed135
1858 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\f2456e06ec69aee6
1859 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\130689708f56d92c
1860 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\a5a6b0abfdf045f7
1861 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\32b33d204044cde0
1862 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\1052cbdce2a30e08
1863 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\8af9f1745f30f393
1864 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\39b386fd4391f8e7
1865 - D:\dnmarchives\dnmarchives\anarchia\2015-07-05\index.php\image\627d27b838f11750
1866 - D:\dnmarchives\dnmarchives\anarchia\